In [1]:
import src
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact
import pyaldata

sns.set_context('talk')

%load_ext autoreload
%autoreload 2

In [3]:
trial_data = src.data.load_clean_data('../data/trial_data/Prez_20220718_RTTCST_TD.mat',keep_unsorted=True,verbose=True)
td = trial_data
td_binned = pyaldata.combine_time_bins(trial_data,100)

Removed 1 trials that monkey aborted
Removed 0 neurons from MC_spikes.


In [6]:
@interact(trial_id=td['trial_id'])
def inspect_rasters(trial_id):
    trial_binned = td_binned.loc[td_binned['trial_id']==trial_id,:].squeeze()
    trial_unbinned = td.loc[td['trial_id']==trial_id,:].squeeze()
    fig,ax=plt.subplots(2,1,figsize=(6,4),sharex=True)
    src.plot.make_trial_raster(trial_binned,ax[0],sig='MC_spikes',events=['idx_goCueTime'],ref_event_idx=trial_binned['idx_goCueTime'])
    src.plot.make_trial_raster(trial_unbinned,ax[1],sig='MC_spikes',events=['idx_goCueTime'],ref_event_idx=trial_unbinned['idx_goCueTime'])


interactive(children=(Dropdown(description='trial_id', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15,…

In [8]:
from sklearn.decomposition import PCA
import numpy as np

model = PCA(n_components=1)
td_temp = pyaldata.dim_reduce(td_binned,model,signal='MC_spikes',out_fieldname='M1_pca')

td_temp = td_temp.explode(['M1_pca','MC_spikes']).explode('M1_pca').reset_index(drop=True)
print(td_temp.loc[td_temp['M1_pca']>40,'trial_id'])
td_temp.loc[td_temp['M1_pca']>40,'MC_spikes'].values[0]/td_temp.iloc[0]['bin_size']

381        5
382        5
383        5
384        5
385        5
386        5
757       11
758       11
759       11
760       11
761       11
2607      36
4855      65
4856      65
4857      65
4858      65
20889    264
20890    264
22217    283
22218    283
22219    283
22220    283
22221    283
22222    283
22223    283
22224    283
22958    294
22959    294
22960    294
22961    294
22962    294
22963    294
22964    294
22965    294
22966    294
22967    294
23181    297
23182    297
23183    297
23184    297
23185    297
23186    297
23187    297
23188    297
23189    297
23190    297
23191    297
23192    297
23193    297
23194    297
27551    351
27552    351
27553    351
27554    351
27555    351
27556    351
27557    351
27558    351
Name: trial_id, dtype: int32


array([ 90., 110., 130.,  90., 120.,  40.,  70.,  70.,  50.,  70.,  40.,
        40.,  40., 120.,  90.,  30.,  40.,  10.,  40.,  20.,  80.,  50.,
        50.,  90.,  70.,  60.,  50.,  80., 100., 100., 100.,  90.,  20.,
        70.,  50., 180.,  30., 100., 120.,  50.,  50.,  20., 180.,  70.,
       110., 140., 100.,  90.,  90.,  80.,  50.,  90.,  70.,  40.,  60.,
        70.,  60.,  90.,  60.,  80.,  80.,  50.,  50.,  60.,  70., 110.,
        40.,  20.,  40.,  60.,  60.,  40.,  20.,  30.,  40.,  10.,  60.,
        20.,  40.,  30.,  30.,  40.,  40.,  70.,  50.,  10.,  20.,  10.,
        30.,  50.])

In [13]:
td_dropped = src.data.remove_bad_trials(td_binned,rate_thresh=350,verbose=True)

6 trials with high firing rates removed. Dropping trials with IDs:
[297   5 283 351  65 294]
